<a href="https://colab.research.google.com/github/hariomshahu/FightDetection/blob/EfficientNetV2B0_3DCNN_For_TemporalInfo/fight_detection_using_EfficientNetV2B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install kaggle

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp "/content/drive/MyDrive/Colab Notebooks/kaggle_api_credentials/kaggle.json" ~/.kaggle/kaggle.json

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d vulamnguyen/rwf2000/

Dataset URL: https://www.kaggle.com/datasets/vulamnguyen/rwf2000/versions/
License(s): unknown
100% 11.5G/11.5G [08:40<00:00, 21.4MB/s]
100% 11.5G/11.5G [08:40<00:00, 23.7MB/s]


In [7]:
import os

dataset_zip = "rwf2000.zip"

destination_folder = "/content/dataset"

os.makedirs(destination_folder, exist_ok=True)

!unzip -q $dataset_zip -d $destination_folder

error:  cannot create /content/dataset/RWF-2000/train/Fight/1060314╤ДтХХ╨╜╤Е╨│╨▓╤Е╨┤╨╣╤Ж╨й╨п╤Й╨Ц╨╗╤Й╨й╨▓╤Ж╨Р╨╡╤И╨╕тХС╤Е╨╛╨┤╤Ж╨ктФд╤Е╨Ъ╨л╤ДтХС╨Ы╤ДтХЧтХв╤ПтХЭ╨Ш╤Е╨Э╨о╤Й╨Ч╨б╤ЕтФВтЦС╤ПтХЭ╨Щ_17.avi
        File name too long
error:  cannot create /content/dataset/RWF-2000/train/Fight/╤ДтХХ╨┤╤Е╨╡тФВ╤Е╨╜╨а╤ЕтХС╨з╤Й╨Ч╨Ь╤Ж╨Щ╨г╤Ж╨отХв╤ПтХЭ╨Ь╤Е╨ЭтФд╤ЕтХЭ╨е╤Е╨Я╨б╤Ж╨етЦС╤Е╨а╨Э╤З╨дтХЦ╤Е╨╜╨а╤Е╨ЧтХС╤Ж╨Щ╨Ы╤ПтХЭ╨Ь╤Ж╨и╨┐╤ДтХЧ╨Р╤ДтХг╨Ш╤Е╨Ю╨п╤Е╨л╨░╤ПтХЭ╨п╤З╨л╨б╤Ж╨Ю╨╖╤Ж╨Ы╨Э╤ДтХХ╨Ы╤Е╨Х╨╕╤З╨╕╨Ы_urlgot_220.avi
        File name too long
error:  cannot create /content/dataset/RWF-2000/train/Fight/╤ДтХХ╨┤╤Е╨╡тФВ╤Е╨╜╨а╤ЕтХС╨з╤Й╨Ч╨Ь╤Ж╨Щ╨г╤Ж╨отХв╤ПтХЭ╨Ь╤Е╨ЭтФд╤ЕтХЭ╨е╤Е╨Я╨б╤Ж╨етЦС╤Е╨а╨Э╤З╨дтХЦ╤Е╨╜╨а╤Е╨ЧтХС╤Ж╨Щ╨Ы╤ПтХЭ╨Ь╤Ж╨и╨┐╤ДтХЧ╨Р╤ДтХг╨Ш╤Е╨Ю╨п╤Е╨л╨░╤ПтХЭ╨п╤З╨л╨б╤Ж╨Ю╨╖╤Ж╨Ы╨Э╤ДтХХ╨Ы╤Е╨Х╨╕╤З╨╕╨Ы_urlgot_221.avi
        File name too long
error:  cannot create /content/dataset/RWF-2000/train/Fight/╤Е╨а╨Ь╤И╨▒╨Ь╤И╨Р╨С╤Ж╨нтФР╤Е╨Я╨╗╤Ж╨н╨╡╤ЕтХХ╨╛╤ЖтФдтХЫ╤Ж╨Щ╨г╤Ж╨отХв╤ПтХЭ╨Ь╤ДтХХ╨Ы╤ДтХХ╨Р╤З╨╖╨в╤Ж╨м╨Щ╤ДтХСтХС╤И╨╖╨С╤Ж╨╜тХЧ╤З╨╡╨

In [8]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [9]:
def prepare_data(data_dir):
    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'val')

    train_fight_dir = os.path.join(train_dir, 'Fight')
    train_nonfight_dir = os.path.join(train_dir, 'NonFight')

    train_fight_videos = [os.path.join(train_fight_dir, f) for f in os.listdir(train_fight_dir)]
    train_nonfight_videos = [os.path.join(train_nonfight_dir, f) for f in os.listdir(train_nonfight_dir)]

    train_paths = train_fight_videos + train_nonfight_videos
    train_labels = [1] * len(train_fight_videos) + [0] * len(train_nonfight_videos)

    val_fight_dir = os.path.join(val_dir, 'Fight')
    val_nonfight_dir = os.path.join(val_dir, 'NonFight')

    val_fight_videos = [os.path.join(val_fight_dir, f) for f in os.listdir(val_fight_dir)]
    val_nonfight_videos = [os.path.join(val_nonfight_dir, f) for f in os.listdir(val_nonfight_dir)]

    val_paths = val_fight_videos + val_nonfight_videos
    val_labels = [1] * len(val_fight_videos) + [0] * len(val_nonfight_videos)

    train_generator = VideoDataGenerator(
        train_paths,
        train_labels,
        batch_size=CONFIG['batch_size'],
        num_frames=CONFIG['num_frames'],
        img_size=CONFIG['img_size'],
        is_training=True
    )

    val_generator = VideoDataGenerator(
        val_paths,
        val_labels,
        batch_size=CONFIG['batch_size'],
        num_frames=CONFIG['num_frames'],
        img_size=CONFIG['img_size'],
        is_training=False
    )

    return train_generator, val_generator

In [10]:
# Configuration
CONFIG = {
    'seed': 42,
    'img_size': 224,
    'num_frames': 32,
    'batch_size': 8,
    'epochs': 50,
    'learning_rate': 1e-4
}

class VideoDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, video_paths, labels, batch_size=8, num_frames=32, img_size=224, is_training=True, **kwargs):
        super().__init__(**kwargs)
        self.video_paths = video_paths
        self.labels = labels
        self.batch_size = batch_size
        self.num_frames = num_frames
        self.img_size = img_size
        self.is_training = is_training

    def __len__(self):
        return len(self.video_paths) // self.batch_size

    def __getitem__(self, idx):
        batch_videos = self.video_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        # Initialize batch arrays
        X = np.zeros((self.batch_size, self.num_frames, self.img_size, self.img_size, 3))
        y = np.zeros(self.batch_size)

        for i, (video_path, label) in enumerate(zip(batch_videos, batch_labels)):
            frames = self._load_video(video_path)
            X[i] = frames
            y[i] = label

        return X, y

    def _load_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frames = []
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        frame_indices = np.linspace(0, frame_count-1, self.num_frames, dtype=int)

        for idx in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if ret:
                frame = cv2.resize(frame, (self.img_size, self.img_size))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = frame / 255.0
                frames.append(frame)

        cap.release()
        return np.array(frames)






In [11]:
def build_model():
    inputs = layers.Input(shape=(CONFIG['num_frames'], CONFIG['img_size'], CONFIG['img_size'], 3))

    base_model = EfficientNetV2B0(
        include_top=False,
        weights='imagenet',
        input_shape=(CONFIG['img_size'], CONFIG['img_size'], 3)
    )

    base_model.trainable = False

    x = tf.keras.layers.TimeDistributed(base_model)(inputs)

    #3D convolution layers for temporal processing
    x = layers.Conv3D(512, kernel_size=(3, 3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)

    x = layers.Conv3D(256, kernel_size=(3, 3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)

    x = layers.GlobalAveragePooling3D()(x)

    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

In [12]:
def train_model(train_generator, val_generator):
    model = build_model()

    optimizer = tf.keras.optimizers.Adam(learning_rate=CONFIG['learning_rate'])
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC()]
    )

    callbacks = [
        ModelCheckpoint(
            'best_model.keras',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6,
            verbose=1
        )
    ]

    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=CONFIG['epochs'],
        callbacks=callbacks,
    )

    return model, history

In [ ]:

def main():
    tf.random.set_seed(CONFIG['seed'])
    np.random.seed(CONFIG['seed'])

    data_dir = '/content/dataset/RWF-2000'
    train_generator, val_generator = prepare_data(data_dir)

    model, history = train_model(train_generator, val_generator)

    pd.DataFrame(history.history).to_csv('training_history.csv')

    print("Training completed!")

if __name__ == "__main__":
    main()

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.4480 - auc: 0.4141 - loss: 0.9978
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to best_model.keras
197/197 ━━━━━━━━━━━━━━━━━━━━ 2330s 8s/step - accuracy: 0.4482 - auc: 0.4143 - loss: 0.9972 - val_accuracy: 0.5000 - val_auc: 0.5126 - val_loss: 1.0037 - learning_rate: 1.0000e-04
Epoch 2/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.4821 - auc: 0.4549 - loss: 0.7927
Epoch 2: val_accuracy did not improve from 0.50000
197/197 ━━━━━━━━━━━━━━━━━━━━ 1431s 7s/step - accuracy: 0.4821 - auc: 0.4550 - loss: 0.7926 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.7176 - learning_rate: 1.0000e-04
Epoch 3/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.4907 - auc: 0.4235 - loss: 0.7409
Epoch 3: val_accuracy did not improve from 0.50000
197/197 ━━━━━━━━━━━━━━━━━━━━ 1431s 7s/step - accuracy: 0.4907 - auc: 0.4237 - loss: 0.7408 - val_accuracy: